# Inlämning 1 av 2
Modellering av Mnist data samt svar på teoretiska frågor.

## Teoretiska frågor

1. Lotta delar upp sin data i ”Träning”, ”Validering” och ”Test”, vad används respektive del för?
    - Träning används för att träna olika modeller för att testa vilket typ av algoritm som matchar data bäst.
    - Valideringsdatan används för att utvärdera de tränade modellerna på data som de inte har influerat modellen under träningen.
    - När valideringen är klar tränar man om den mest lämpliga modellen på på både tränings och valideringdatan för att låta så mycket data som möjligt ligga till grund för modellen. När denna andra träning är klar testar man datan på testdatan och det ger dig det slutgiltiga kvittot på hur bra som modellen kommer att fungera.

2. Förklara (gärna med ett exempel): Ordinal encoding, one-hot encoding, dummy variable
encoding.
    - Ordinal encoding används för kategorier med ordning, varkategori får ett heltal baserat på dess ordningsföljd.
        "Låg" → 0
        "Medel" → 1
        "Hög" → 2
    - One-hot encoding skapar binära variabler för varje kategori, om variabeln finns får den värdet 1, används för kategorier utan rangordning används för andra AI modeller förutom linjär regression.
        "Röd" → [1, 0, 0]
        "Grön" → [0, 1, 0]
        "Blå" → [0, 0, 1]
    - Dummy encoding liknar one-hot men utesluter en referenskategori, "Blå" är referens är värdet noll är kategoring "Blå". Används för linjär regression.
        "Röd" → [1, 0]
        "Grön" → [0, 1]
        "Blå" → [0, 0]

3. Göran påstår att datan antingen är ”ordinal” eller ”nominal”. Julia säger att detta måste
tolkas. Hon ger ett exempel med att färger såsom {röd, grön, blå} generellt sett inte har
någon inbördes ordning (nominal) men om du har en röd skjorta så är du vackrast på festen
(ordinal) – vem har rätt?
    - Det tråkiga svaret är nog bägge beroende på situation. Frågar du 10000 människor om vilken skjorta som är snyggast, en röd eller en blå tror jag inte att resultatet kommer vara 50%, men bägge skjortorna fyller samma syfte och det spelar egentligen ingen roll vilken du har på dig.
    - Jag tycker detta visar att det är viktigt att både förstå datan och tillämpningen. Ska modellen du utvecklar hjälpa ett klädföretag att ta reda på hur många skjortor de ska tillverka av olika färger nästa år så behöver du rangordna färgerna eftersom de förmodligen kommer att sälja olika många. MEn om applikationen använder antalet skjortor folk äger för att mäta vilken klass i samhället du tillhör spelar nog färgen ingen roll. 
    (Krysstat exempel, jag vet, men jag är lite begränsad eftersom det handlar om skjortor)

4. Läs följande länk: https://stackoverflow.com/questions/56107259/how-to-save-a-trainedmodel-by-scikit-learn
(speciellt svaret från användaren som heter ”sentence”) som beskriver ”joblib” och ”pickle”.
Det är alltså ett sätt att spara modeller och innebär att man kan träna en modell och sedan
återanvända den för att göra prediktioner utan att behöva träna om modellen. Detta
kommer ni ha nytta av om ni satsar på VG delen.
Svara på frågan: Vad används joblib och pickle till?
    - Ett satt att spara, packetera, tränade modeller för att kunna återanvända i flera applikationer.

## Modellering av Mnist

In [2]:

import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.datasets import fetch_openml

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier

np.random.seed(42)

In [4]:
# Loading Mniast dataset, and splitting it into train, test, and validation
# I used a total off 10000 images to speed up training and make the finished model smaller
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=2000,train_size= 10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=2000, random_state=42)

In [5]:
# Instatiating the 2 models for training
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

In [6]:
# Training the 2 models on the training data
random_forest_clf.fit(X_train, y_train) 
extra_trees_clf.fit(X_train, y_train)

ExtraTreesClassifier(random_state=42)

In [7]:
# Evaluating the 2 models on the validation data using the score method

print(random_forest_clf.score(X_val, y_val))
print(extra_trees_clf.score(X_val, y_val))

# The extra trees classifier is the winner

0.9435
0.953


In [10]:
# Retrain the best model on the training, and validation data.

extra_trees_clf.fit(X_train_val, y_train_val)

ExtraTreesClassifier(random_state=42)

In [11]:
# Evaluate on the test data

print(extra_trees_clf.score(X_test, y_test))

# Slightly better using 2000 more databpoints
# 0.9545 vs 0.953 for the first model

0.9545


In [12]:
# Save the model for future use

joblib.dump(extra_trees_clf, "et_clf_mnist_10000.pkl")

['et_clf_mnist_10000.pkl']